# Graph Isomorphism

In this chapter we construct a zero-knowledge protocol around graph isomorphism.

This chapter is based on [a lecture from the Max Plank Institute for Informatics](https://resources.mpi-inf.mpg.de/departments/d1/teaching/ss13/gitcs/lecture9.pdf).

# What is a graph? What is isomorphism?

Here, a **graph** consists of nodes and edges. Nodes are points in space and edges are bridges between nodes. We usually assign labels to nodes and edges to better keep track of them.

Two graphs are **isomorphic** if they have the same structure. Imagine I have a graph that shows me each country and the connections (edges) to its neighbors. Now I translate this graph into Japanese. The node labels have changed, but intuitively we know that the map is still the same. The node labels don't matter; what matters is the edges. In this example, the English and Japanese graph are isomorphic, because we can translate between the two.

In general, two graphs are isomorphic if we can produce one from the other by translating their node labels. This translation is a 1-to-1 mapping of node labels and is called a **bijection**. Shuffling node labels is always a bijection.

Given two very large random graphs, it is very hard to know if they are isomorphic. There is no known algorithm to quickly compute this (in polynomial time).

# What is an interactive proof?

Peggy and Victor are engaged in an interactive proof.

There are two (random) graphs.

Peggy thinks she knows a translation between both graphs, ergo that both graphs are isomorphic. She wants to prove that to Victor, without revealing the translation.

Victor is sceptical and wants to see evidence. In particular he doesn't want to be convinced if Peggy is lying (there is no translation).

Peggy wins if she convinces Victor. Victor wins by accepting only graphs that are actually isomorphic.

# Jupyter setup

Run the following snippet to set up your jupyter notebook for the workshop.

In [ ]:
import os
import sys

# Add project root so we can import local modules
root_dir = sys.path.append("..")
sys.path.append(root_dir)

# Graph Parameters

Choose the number of nodes and the number of edges for the first graph.

Feel free to experiment with different values.

In [ ]:
import ipywidgets as widgets
from IPython.display import display

# You can adjust the sliders any time

num_nodes = widgets.IntSlider(min=5, max=30, value=10, step=1, description="#Nodes")
num_edges = widgets.IntSlider(min=5, max=60, value=20, step=1, description="#Edges")

display(num_nodes)
display(num_edges)

## First Graph

Generate the first graph.

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
from local.graph import Mapping, random_graph, non_isomorphic_graph

# Rerun this cell to regenerate the first graph

graph1 = random_graph(num_nodes.value, num_edges.value)
print("First graph")
nx.draw(graph1, with_labels=True)
plt.show()

# Scenario

There are two scenarios in this interactive proof:

1. **Peggy is honest** and knows a translation between both graphs. She hopes to convince Victor of the true statement that both graphs are isomorphic.
2. **Peggy is lying** and doesn't actually know anything! It might even be that both graphs are not isomorphic, which means that there cannot exist a translation between them. Peggy still tries to fool Victor of the false statement that both graphs are isomorphic.

Choose the good or the evil scenario. See how it affects the other cells further down.

In [ ]:
# You can adjust the selection any time
# Make sure to re-generate the second graph afterwards

is_isomorphic = widgets.Dropdown(
    options=[
        ("Second graph isomorphic (honest Peggy 😇)", True),
        ("Second graph not isomorphic (lying Peggy 😈)", False)],
    value=True,
    description="Scenario:",
)
is_isomorphic

# Second Graph

Now generate the second graph. In the good scenario it is isomorphic to the first graph. In the evil scenario, it is not isomorphic.

In [ ]:
# Rerun this cell to generate a new second graph
# Rerun this cell after changing the scenario above

if is_isomorphic.value:
    from_1_to_2 = Mapping.shuffle_graph(graph1)
    graph2 = from_1_to_2.apply_graph(graph1)
    print("Second graph [isomorphic to first graph]")
    print("Translation: {}".format(from_1_to_2))
else:
    graph2 = non_isomorphic_graph(graph1)
    print("Second graph [not isomorphic to first graph]")
    # Unrelated random shuffling (lying Peggy)
    from_1_to_2 = Mapping.shuffle_graph(graph1)
    print("Fake translation: {}".format(from_1_to_2))
    
nx.draw(graph2, with_labels=True)
plt.show()

# Protocol

Now onto the meat; how does this proof work?

Peggy randomly shuffles the graph one and sends the resulting shuffled graph to Victor.

Victor randomly chooses between graph one or two.

For graph one, Peggy sends the random shuffling from before.

For graph two, she sends a more complicated bijection: She translates graph two back to graph one (which is possible if she knows a translation) and then applies the random shuffling on top of that.

Victor translates his chosen graph using the bijection that Peggy sent. He checks if this translated graph is equal the shuffled graph that Peggy sent.

In [ ]:
import random

class Peggy:
    def __init__(self, graph1: nx.Graph, from_1_to_2: Mapping):
        self._graph1 = graph1
        self._from_1_to_2 = from_1_to_2
    
    def shuffled_graph(self) -> nx.Graph:
        self._shuffle = Mapping.shuffle_graph(self._graph1)
        shuffled_graph1 = self._shuffle.apply_graph(self._graph1)
        return shuffled_graph1
    
    def respond(self, index: int) -> Mapping:
        if index == 0:
            return self._shuffle
        else:
            assert index == 1
            complex_shuffle = self._from_1_to_2.invert().and_then(self._shuffle)
            return complex_shuffle


class Victor:
    def __init__(self, graph1: nx.Graph, graph2: nx.Graph):
        self._graphs = [graph1, graph2]
        
    def random_index(self, shuffled_graph: nx.Graph) -> int:
        self._shuffled_graph = shuffled_graph
        self._index = random.randrange(0, 2)
        return self._index
    
    def verify(self, shuffle: Mapping) -> bool:
        another_shuffled_graph = shuffle.apply_graph(self._graphs[self._index])
        # `self._shuffled_graph == another_shuffled_graph` compares pointers not data
        return set(sorted(self._shuffled_graph.edges())) == set(sorted(another_shuffled_graph.edges()))

# Single Exchange

Let's run a couple of exchanges to get familiar. Feel free to experiment with the graph parameters and the scenario. The system is random so there could be a different outcome next time you run with the same settings 😉

In [ ]:
# Feel free to run this multiple times

peggy = Peggy(graph1, from_1_to_2)
victor = Victor(graph1, graph2)

shuffled_graph = peggy.shuffled_graph()
index = victor.random_index(shuffled_graph)
response_shuffle = peggy.respond(index)

# Victor is convinced
if victor.verify(response_shuffle):
    # Two graphs were isomorphic (good)
    if is_isomorphic.value:
        print("Convinced 👌 (expected)")
    # Two graphs were non-isomorphic (evil)
    else:
        print("Convinced 👌 (Victor was fooled)")
# Victor is not convinced
else:
    # Two graphs were isomorphic (good)
    if is_isomorphic.value:
        print("Not convinced... 🤨 (Peggy was dumb)")
    # Two graphs were non-isomorphic (evil)
    else:
        print("Not convinced... 🤨 (expected)")

# Completeness

Does Peggy have a chance to convince Victor if she is honest? Let's run the protocol a couple of times and see how often she manages to convince him.

In [ ]:
num_exchanges_complete = widgets.IntSlider(min=10, max=1000, value=10, step=10, description="#Exchanges")
num_exchanges_complete

In [ ]:
from_1_to_3 = Mapping.shuffle_graph(graph1)
graph3 = from_1_to_3.apply_graph(graph1)

honest_peggy = Peggy(graph1, from_1_to_3)
victor = Victor(graph1, graph3)

peggy_success = 0

for _ in range(num_exchanges_complete.value):
    shuffled_graph = honest_peggy.shuffled_graph()
    index = victor.random_index(shuffled_graph)
    response_shuffle = honest_peggy.respond(index)

    if victor.verify(response_shuffle):
        peggy_success += 1
        
peggy_success_rate = peggy_success / num_exchanges_complete.value * 100

print("Running {} exchanges".format(num_exchanges_complete.value))
print("Honest Peggy wins {0:0.2f}% of the time".format(peggy_success_rate))
print()

assert peggy_success_rate == 100
print("Peggy always wins if she is honest")

# Soundness

Now, does Victor have a chance to expose Peggy if she is lying? This is not so easy, because in the original protocol there is actually a 50% chance that Victor is fooled!

He challenges Peggy to provide a bijection based on a graph he chooses. If he chooses graph one, then Peggy can send him the random shuffling she generated. Anyone can do that! If he chooses graph two, then Peggy has to translate between graph one and two; that is the complicated part. She cannot do that unless she really knows a translation.

There is an easy way to boost Victor's confidence: run the protocol for **multiple rounds**. Peggy randomly shuffles graph one, Victor chooses between graph one or two, Peggy responds; repeat. Each time the random values will be different. See how Victor gets to challenge Peggy once per round. If Peggy ever fails to meet Victor's challenge, then Victor rejects. The game is asymmetric: Peggy has to perform **every time** while Victor has to expose her cheating **only once**. He has a much easier time and thus gains in confidence.

The chance that Victor gets fooled after `n` rounds is `0.5^i`, which decreases exponentially in `i`.

In [ ]:
num_exchanges_sound = widgets.IntSlider(min=10, max=1000, value=10, step=10, description="#Exchanges")
num_rounds = widgets.IntSlider(min=1, max=10, value=1, step=1, description="#Rounds")

display(num_exchanges_sound)
display(num_rounds)

In [ ]:
graph4 = non_isomorphic_graph(graph1)
# Random shuffle that does not translate between graphs 1 and 4
from_1_to_4 = Mapping.shuffle_graph(graph1)

lying_peggy = Peggy(graph1, from_1_to_4)
victor = Victor(graph1, graph4)

victor_success = 0

for _ in range(num_exchanges_sound.value):
    for _ in range(num_rounds.value):
        shuffled_graph = lying_peggy.shuffled_graph()
        index = victor.random_index(shuffled_graph)
        response_shuffle = lying_peggy.respond(index)
    
        if not victor.verify(response_shuffle):
            victor_success += 1
            break
            
victor_success_rate = victor_success / num_exchanges_sound.value * 100

print("Running {} exchanges with {} rounds each".format(num_exchanges_sound.value, num_rounds.value))
print("Victor wins against lying Peggy {0:0.2f}% of the time".format(victor_success_rate))
print()

if victor_success_rate < 50:
    print("Victor loses quite often for a small number of rounds")
elif victor_success_rate < 90:
    print("Victor gains more confidence with each added round")
else:
    print("At some point it is basically impossible to fool Victor")

# Zero-Knowledge

Does Victor learn anything about the secret translation from his exchange with Peggy?

Peggy sends the shuffled version of one of the two graphs. Victor sends an index. Peggy sends a bijection that can be applied to the graph at Victor's index to produce the shuffled graph that she originally sent.

It might seem complicated, but we can easily replicate the same behavior if we switch the order of steps:

First we choose an index. Then we shuffle the graph at that index and reveal the resulting graph as well as the shuffling. Because all shufflings in the protocol have some inbuilt randomness, our fake transcripts should look real.

Let's do a chi-square test to see if the real and fake transcripts follow the same distribution.

In [ ]:
num_transcripts = widgets.IntSlider(min=1000, max=50000, value=10000, step=1000, description="#Transcripts")
num_transcripts

In [ ]:
from typing import Tuple
import local.stats as stats

# Make sure to run this for small graphs (adjust the graph parameters)
# Large graphs can lead to errors
# The list of critical chi-square values might be too short for the large number of degrees of freedom
# Large graphs also require very many transcripts, which is slow

peggy = Peggy(graph1, from_1_to_2)
victor = Victor(graph1, graph2)

def real_transcript() -> Tuple:
    shuffled_graph = peggy.shuffled_graph()
    index = victor.random_index(shuffled_graph)
    response_shuffle = peggy.respond(index)

    return tuple(shuffled_graph.edges()), index, tuple(response_shuffle)


def fake_transcript() -> Tuple:
    index = random.randrange(0, 2)

    if index == 0:
        shuffle = Mapping.shuffle_graph(graph1)
        shuffled_graph = shuffle.apply_graph(graph1)
        response_shuffle = shuffle
    else:
        shuffle = Mapping.shuffle_graph(graph2)
        shuffled_graph = shuffle.apply_graph(graph2)
        response_shuffle = shuffle

    return tuple(shuffled_graph.edges()), index, tuple(response_shuffle)


real_samples = [real_transcript() for _ in range(num_transcripts.value)]
fake_samples = [fake_transcript() for _ in range(num_transcripts.value)]

# The chi-square test is only valid if most bins are filled
# Increase the number of transcripts if there are too many empty bins

null_hypothesis = stats.chi_square_equal(real_samples, fake_samples)
print()

if null_hypothesis:
    print("Real and fake transcripts are the same distribution.")
    print("Victor learns nothing 👌")
else:
    print("Real and fake transcripts are different distributions.")
    print("Victor might learn something 😧")

stats.plot_comparison(real_samples, fake_samples, "real", "fake")